### Implement a Deep Neural Network

In [1]:
import torch
import torch.nn as nn
from matplotlib import pyplot as plt

In [26]:
# Generate synthetic data
torch.manual_seed(42)
X = torch.rand(100, 2) * 10  # 100 data points with 2 features
y = (X[:, 0] + X[:, 1] * 2).unsqueeze(1) + torch.randn(100, 1)  # Non-linear relationship with noise

In [27]:
class CustomDNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )
    def forward(self, x):
        x = self.model(x)
        return x

In [28]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X[:, 0])
    
    def __getitem__(self, index):
        return self.X[index, :], self.y[index]
    
class CustomLoss(nn.Module):
    def __init__(self, beta):
        super().__init__()
        self.beta = beta
        
    def forward(self, pred, y):
        dev = pred - y
        map_function = dev.abs() <= self.beta
        loss = torch.where(map_function, 0.5*(dev**2), self.beta*dev.abs() - 0.5*self.beta) 
        return loss.mean()

In [29]:
dataset = CustomDataset(X, y)
print(dataset.__getitem__(3))

(tensor([2.5657, 7.9364]), tensor([18.2536]))


In [30]:
dataset = CustomDataset(X, y)
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=100)
model = CustomDNN(2, 8)
criterion = nn.MSELoss()
custom_criterion = CustomLoss(1.0)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [31]:
epochs = 1000
print(X.shape, y.shape)
for epoch in range(epochs):
    
    for X_train, y_train in train_dataloader:
        # print(X_train.shape, y_train.shape)
        pred = model(X_train)
        loss = criterion(pred, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch %100 == 0:
        print(f"Loss = {loss}")


torch.Size([100, 2]) torch.Size([100, 1])
Loss = 306.88970947265625
Loss = 0.7754282355308533
Loss = 0.7585095167160034
Loss = 0.7529838681221008
Loss = 0.7483873963356018
Loss = 0.7445487976074219
Loss = 0.7413382530212402
Loss = 0.7386481761932373
Loss = 0.7363910675048828
Loss = 0.7344982028007507


In [32]:
X_test = torch.tensor([[4.0, 3.0], [7.0, 8.0]])
with torch.no_grad():
    predictions = model(X_test)
    print(f"Predictions for {X_test.tolist()}: {predictions.tolist()}")

Predictions for [[4.0, 3.0], [7.0, 8.0]]: [[9.821194648742676], [23.15705680847168]]


## Bonus: view(), squeeze() and unsqueeze() problems
### Descriptions
1. **view()**
Reshapes a tensor without changing its underlying data.
Key point: The new shape must be compatible with the original number of elements.
For example, a tensor with shape (2, 3) has 6 elements. You could view it as (3, 2), (6), (1, 6), etc., because all these target shapes contain exactly 6 elements.

2. **squeeze()**
Removes all dimensions of size 1 from the tensor shape.
Eg: If you have a tensor of shape (1, 3, 1, 4), squeezing it will yield a shape (3, 4).

3. **unsqueeze()**
Adds a dimension of size 1 at a specified index.
Eg: If you have a tensor of shape (3, 4) and you unsqueeze(dim=0), you get (1, 3, 4). If you unsqueeze(dim=2), you get (3, 4, 1).


In [39]:
a = torch.randn(2, 3)
print(f"original: {a}")
print(f"view: {a.view(3, 2)}, {a.view(-1)}")
print(f"transpose: {a.T}") #Transpose is different from view

original: tensor([[ 0.3804, -0.3900,  0.9222],
        [ 0.7472, -2.1167, -0.9752]])
view: tensor([[ 0.3804, -0.3900],
        [ 0.9222,  0.7472],
        [-2.1167, -0.9752]]), tensor([ 0.3804, -0.3900,  0.9222,  0.7472, -2.1167, -0.9752])
transpose: tensor([[ 0.3804,  0.7472],
        [-0.3900, -2.1167],
        [ 0.9222, -0.9752]])


In [44]:
b = torch.randn((1, 3, 1, 2))
print(f"original: {b.shape}")
print(f"squeeze all: {b.squeeze().shape}")
print(f"partial squeeze at dim=2: {b.squeeze(dim=2).shape}")
print(f"unsqueeze at dim 0: {b.unsqueeze(dim=0).shape}")


original: torch.Size([1, 3, 1, 2])
squeeze all: torch.Size([3, 2])
partial squeeze at dim=2: torch.Size([1, 3, 2])
unsqueeze at dim 0: torch.Size([1, 1, 3, 1, 2])
